# Dados coletados do CovidHub

https://covid19forecasthub.org/doc/ensemble/
## Sobre os Dados
No Covid-Hub, forma feitos modelos para casos e mortes, com forecasts por estado,
sendo estes agregados para nível nacional. Neste notebook utilizamos os dados de nível nacional (US).

Os dados reais (ground truth) de Covid se encontram no dataframe `covid_data`,
enquanto `forecasts` contém os modelos de forecasting (ensembles e modelos componentes utlizados no ensamble).

Forecasts podem conter quantis ou point predictions, que está na coluna `type`. Aqui usamos somente os quantis, que
foram os valores utilizados para o ensamble.

### Modelos

Existem diversos modelos na tabela de `forecasts`.
#### COVIDhub-4_week_ensemble

"This ensemble produces forecasts of incident cases (discontinued as of February 2023), incident deaths, and cumulative deaths (discontinued as of March 2023) at horizons of 1 through 4 weeks ahead, and forecasts of incident hospitalizations at horizons of 1 through 28 days ahead. For all of these targets, the ensemble forecasts are computed as the equally-weighted median of all component forecasts at each location, forecast horizon, and quantile level."

In [1]:
import pandas as pd
import polars as pl
import polars.selectors as cs

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scoringrules as sr

from lets_plot import *
LetsPlot.setup_html()

#### Ground Truth

Os dados de ground truth são simples. A tabela tem `value` para os valores e `target_end_date` como a data
da medição.

In [2]:
covid_data = pl.read_parquet('./data/covid_truth_data.parquet')
covid_data = covid_data.with_columns(
    pl.lit('ground_truth').alias('type'),
    pl.col('target_end_date').cast(pl.Datetime),
    pl.col('target_end_date').dt.weekday().alias('target_day_of_week')
)


In [3]:
(
    ggplot(data=covid_data) +
    geom_line(aes(x='target_end_date', y='value')) +
    geom_point(aes(x='target_end_date', y='value')) +
    ggsize(1400, 400)+
    theme_bw()
)

#### Forecasts

Na tabela de forecast temos os modelos, o valor previsto para um certo `horizon` e um certo `quantile`.
Cada forecast tem um `forecast_date` que é quando o forecast foi submetido, e tem um `target_end_date`,
que é a data que está sendo prevista (por exemplo, para forecast_date de 1/1/2020 eu prevejo o número
de caso do target_end_date 8/1/2020).

Temos diversos modelos, porém, somente os modelos 'primary' são utilizados no ensamble.
Além disso, há modelos que são submetidos mais de uma vez, de forma que apresentam
duas datas de 'forecast_date' para um mesmo quantil, horizon e target_end_date. A razão disso pode ter sido
uma correção, ou erro de submissão. De toda forma, o ensamble usa a última submissão.

Assim, nossa tabela de `forecasts` será filtrada para o `forecast_date` que caem na Segunda, e filtraremos
também somente modelos primary.

In [4]:
models = pl.read_parquet('./data/models.parquet')

models = models.with_columns(
    (pl.col('designation') == 'primary').alias('include_ensemble')
)

# models = models.filter(
#     pl.col('designation') == 'primary'
# )

models = models.with_columns(
    pl.when(pl.col('model').is_in(['COVIDhub-4_week_ensemble', 'COVIDhub-trained_ensemble', 'COVIDhub-baseline']))
    .then(pl.lit('primary'))
    .otherwise(pl.col('designation'))
    .alias('designation')
)


In [5]:
models

model,designation,include_ensemble
str,str,bool
"""KITmetricslab-select_ensemble""","""primary""",true
"""CovidAnalytics-DELPHI""","""primary""",true
"""UMass-MechBayes""","""primary""",true
"""UT-Osiris""","""primary""",true
"""USACE-ERDC_SEIR""","""primary""",true
…,…,…
"""MIT-Cassandra""","""primary""",true
"""PandemicCentral-COVIDForest""","""primary""",true
"""JHU_IDD-CovidSP""","""primary""",true


In [6]:
forecast = pl.read_parquet('./data/fullforecasts.parquet')
forecast = forecast.with_columns(
    pl.col('forecast_date').cast(pl.Datetime),
    pl.col('target_end_date').cast(pl.Datetime),
    pl.col('target_end_date').dt.weekday().alias('target_day_of_week'),
    pl.col('forecast_date').dt.weekday().alias('forecast_day_of_week')
)
# ).filter(
#     pl.col('forecast_day_of_week') == 1,
# )

forecast = forecast.join(models, on='model', how='left').filter(
    pl.col('designation') == 'primary'
)
latest_dates = (
    forecast
    .group_by(['model', 'quantile', 'target_end_date', 'horizon'])
    .agg(pl.col('forecast_date').max().alias('forecast_date'))
)

forecast = forecast.join(latest_dates, on=['model', 'quantile', 'target_end_date', 'horizon','forecast_date'], how='inner')

In [7]:
forecast.filter(
    pl.col('model') == 'BPagano-RtDriven'
).sort('forecast_date').head(10)

model,forecast_date,location,horizon,temporal_resolution,target_variable,target_end_date,type,quantile,value,location_name,population,geo_type,geo_value,abbreviation,full_location_name,target_day_of_week,forecast_day_of_week,designation,include_ensemble
str,datetime[μs],str,str,str,str,datetime[μs],str,f64,f64,str,f64,str,str,str,str,i8,i8,str,bool
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.025,252157.46981,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.1,309027.43446,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.25,360710.91108,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.5,418324.95041,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.75,475938.98974,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.9,527622.46637,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""1""","""wk""","""inc case""",2020-10-24 00:00:00,"""quantile""",0.975,584492.43102,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""2""","""wk""","""inc case""",2020-10-31 00:00:00,"""quantile""",0.025,250557.3476,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true
"""BPagano-RtDriven""",2020-10-18 00:00:00,"""US""","""2""","""wk""","""inc case""",2020-10-31 00:00:00,"""quantile""",0.1,325076.2206,"""United States""",3.32875137e8,"""state""","""us""","""US""","""United States""",6,7,"""primary""",true


# Visualizando os Dados

In [8]:
def quantile_plot(forecast, ground_truth):
    quantile_df  = forecast.pivot(
        index=['target_end_date', 'horizon'],
        on='quantile',
        values='value'
    )
    return (
        ggplot(quantile_df, aes(x='target_end_date')) +
        geom_ribbon(aes(ymin='0.025', ymax='0.975'), fill='#084594', alpha=0.1,size=0.2,manual_key='2.5% to 97.5%') +
        geom_ribbon(aes(ymin='0.25', ymax='0.75'), fill='#2171b5', alpha=0.3,size=0.2,manual_key='25% to 75%') +
        geom_line(aes(y='0.5'), color='blue', size=0.4, manual_key='Median (0.5)') +
        facet_grid(y='horizon', scales='fixed') +
        geom_line(aes(x='target_end_date', y='value'), data=ground_truth, color='black',size=0.5, linetype='dashed', manual_key='Ground Truth') +
        ggsize(1200, 800) +
        theme_bw() +
        labs(title='Forecast Quantiles as Ribbons', y='Value', x='Date')
        )
    

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output

model_options = forecast.select('model').unique().to_series().sort().to_list()
model_dropdown = widgets.Dropdown(
    options=model_options,
    value='COVIDhub-4_week_ensemble',
    description='Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

def update_plot(change):
    with output:
        clear_output(wait=True)
        selected_model = change['new']
        display(quantile_plot(forecast.filter(pl.col('model') == selected_model), covid_data))

model_dropdown.observe(update_plot, names='value')

display(model_dropdown)
with output:
    display(quantile_plot(forecast.filter(pl.col('model') == model_dropdown.value), covid_data))
display(output)

Dropdown(description='Model:', index=2, layout=Layout(width='50%'), options=('BPagano-RtDriven', 'CEID-Walk', …

Output()

## Computing WIS


In [10]:
fs = forecast.pivot(
    values='value',
    index=['model', 'forecast_date', 'target_end_date', 'horizon'],
    on='quantile'
)

fs = fs.join(covid_data[['target_end_date','value']],on=['target_end_date'],how='left')

In [11]:
fs.head()

model,forecast_date,target_end_date,horizon,0.025,0.1,0.25,0.5,0.75,0.9,0.975,value
str,datetime[μs],datetime[μs],str,f64,f64,f64,f64,f64,f64,f64,f64
"""BPagano-RtDriven""",2020-10-18 00:00:00,2020-10-24 00:00:00,"""1""",252157.46981,309027.43446,360710.91108,418324.95041,475938.98974,527622.46637,584492.43102,485474.0
"""BPagano-RtDriven""",2020-10-18 00:00:00,2020-10-31 00:00:00,"""2""",250557.3476,325076.2206,392799.0415,468292.9038,543786.76609,611509.58699,686028.46,572162.0
"""BPagano-RtDriven""",2020-10-18 00:00:00,2020-11-07 00:00:00,"""3""",236909.92449,332846.6848,420034.10156,517226.0783,614418.05504,701605.47179,797542.2321,777428.0
"""BPagano-RtDriven""",2020-10-18 00:00:00,2020-11-14 00:00:00,"""4""",213876.19851,330803.52595,437067.18958,555524.36963,673981.54967,780245.2133,897172.54074,1.028914e6
"""BPagano-RtDriven""",2020-10-25 00:00:00,2020-10-31 00:00:00,"""1""",299044.13571,364922.81525,424793.42033,491534.04269,558274.66504,618145.27012,684023.94966,572162.0


In [12]:
def compute_wis(quantiles, y, taus):
    """
    - quantiles (list): List of predictive quantiles q_1 to q_K
    - y (float): Observed quantity
    - taus (list): List of corresponding tau values (e.g., [0.025, 0.25, 0.5, 0.75, 0.975])
    
    Returns:
        float: WIS value
    """
    K = len(quantiles)
    wis = 0
    for k in range(K):
        indicator = 1 if y <= quantiles[k] else 0
        wis += 2 * (indicator - taus[k]) * (quantiles[k] - y)
    return wis / K
    
def maybe_compute_wis(quantiles, y, taus):
    try:
        return compute_wis(quantiles, y, taus)
    except Exception:
        return np.nan

        
taus = [0.025,0.1,0.25,0.5,0.75,0.9,0.975]
def row_wis(row,taus):
    quantiles = [row[str(t)] for t in taus]
    y = row['value']
    return maybe_compute_wis(quantiles, y, taus)

In [13]:
fs = fs.with_columns(
    pl.struct(pl.all()).map_elements(lambda r: row_wis(r,taus),return_dtype=float).alias('wis')
)

## Model Ensamble - Untrained

*Non-Robust untrained*: Use the mean of each quantile prediction in each component forecast

*Robust untrained*: Use the median of each quantile prediction in each component forecast


Let us create a dataframe with the mean and median of each quantile prediction in each component forecast.

In [14]:
components = fs.join(models.filter('include_ensemble'),on='model',how='inner') #filter only models that are included in the ensemble

# Compute the mediana and mean
components = components.group_by(['target_end_date','horizon']).agg(
    [pl.col(str(q)).mean().alias('mean_' + str(q)) for q in taus] +
    [pl.col(str(q)).median().alias('median_' + str(q)) for q in taus]+
    [pl.col('value').first()]
).sort('target_end_date')

# Adjust the dataframe format
components = components.unpivot(
    index=['target_end_date','horizon'],
    on   = ['mean_'+str(q) for q in taus] + ['median_'+str(q) for q in taus]
).with_columns(
    pl.col('variable').str.split('_').map_elements(lambda x: float(x[1]) if len(x) > 1 else None,return_dtype=float).alias('quantile'),
    pl.col('variable').str.split('_').map_elements(lambda x: 'nonrobust_untrained' if x[0] == 'mean' else 'robust_untrained',return_dtype=str).alias('model')
).sort(['model','target_end_date','horizon','quantile'])

In [15]:
components

target_end_date,horizon,variable,value,quantile,model
datetime[μs],str,str,f64,f64,str
2020-07-11 00:00:00,"""1""","""mean_0.025""",255153.039693,0.025,"""nonrobust_untrained"""
2020-07-11 00:00:00,"""1""","""mean_0.1""",270625.425985,0.1,"""nonrobust_untrained"""
2020-07-11 00:00:00,"""1""","""mean_0.25""",285316.477333,0.25,"""nonrobust_untrained"""
2020-07-11 00:00:00,"""1""","""mean_0.5""",315731.62313,0.5,"""nonrobust_untrained"""
2020-07-11 00:00:00,"""1""","""mean_0.75""",325440.206866,0.75,"""nonrobust_untrained"""
…,…,…,…,…,…
2022-04-09 00:00:00,"""4""","""median_0.25""",64247.123504,0.25,"""robust_untrained"""
2022-04-09 00:00:00,"""4""","""median_0.5""",125651.100316,0.5,"""robust_untrained"""
2022-04-09 00:00:00,"""4""","""median_0.75""",187055.077128,0.75,"""robust_untrained"""


Let us now compare our ensamble with the one that comes directly from the CovidHub dataset. 
Note that the CovidHub only has the `COVIDhub-4_week_ensemble`, which is the robust untrained ensemble.
The other ensemble is the `COVIDhub-trained_ensemble`, which is the robust trained.

In [16]:
covidhub_robust_untrained = fs.sort('target_end_date').filter(
    pl.col('model') == 'COVIDhub-4_week_ensemble'
)

covidhub_robust_untrained = covidhub_robust_untrained.rename({'value':'y'})\
    .unpivot(
        on=[str(t) for t in taus],
        index=["target_end_date",'model','horizon','forecast_date','wis','y'])\
    .rename({'variable':'quantile'})\
    .with_columns(
        pl.col('quantile').cast(pl.Float64)
    ).sort(['model','target_end_date','horizon','quantile'])


In [17]:
comparison = covidhub_robust_untrained.join(components.filter(pl.col('model') =='robust_untrained')[['target_end_date','horizon','quantile','value']],
                                    on=['target_end_date','horizon','quantile'])

comparison = comparison.with_columns(
    (pl.col('value')- pl.col('value_right')).alias('diff'),
)

comparison = comparison.with_columns(
    (pl.col('diff')/pl.col('value')).alias('relative_diff')
)

In [18]:
comparison.sort('relative_diff')

target_end_date,model,horizon,forecast_date,wis,y,quantile,value,value_right,diff,relative_diff
datetime[μs],str,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64
2022-03-26 00:00:00,"""COVIDhub-4_week_ensemble""","""2""",2022-03-14 00:00:00,24074.014286,210000.0,0.025,39215.0,61029.906,-21814.906,-0.55629
2022-03-26 00:00:00,"""COVIDhub-4_week_ensemble""","""2""",2022-03-14 00:00:00,24074.014286,210000.0,0.1,62065.0,95312.424049,-33247.424049,-0.535687
2022-04-02 00:00:00,"""COVIDhub-4_week_ensemble""","""3""",2022-03-14 00:00:00,33983.485714,198718.0,0.1,37424.0,53599.43,-16175.43,-0.432221
2020-08-15 00:00:00,"""COVIDhub-4_week_ensemble""","""4""",2020-07-20 00:00:00,45018.314286,391741.0,0.025,224612.0,320241.890716,-95629.890716,-0.425756
2022-03-12 00:00:00,"""COVIDhub-4_week_ensemble""","""4""",2022-02-14 00:00:00,61613.792857,237276.0,0.1,94027.0,125346.24246,-31319.24246,-0.333088
…,…,…,…,…,…,…,…,…,…,…
2022-03-05 00:00:00,"""COVIDhub-4_week_ensemble""","""4""",2022-02-07 00:00:00,127889.557143,337025.0,0.1,237206.0,168920.624691,68285.375309,0.287874
2022-04-02 00:00:00,"""COVIDhub-4_week_ensemble""","""4""",2022-03-07 00:00:00,30210.278571,198718.0,0.5,179928.0,127436.203714,52491.796286,0.291738
2022-02-12 00:00:00,"""COVIDhub-4_week_ensemble""","""4""",2022-01-17 00:00:00,1.5932e6,1.259905e6,0.025,1.648119e6,1049378.5,598740.5,0.363287


In [19]:
(
    # ggplot(data=comparison.filter(pl.col('horizon')=='1',pl.col('quantile')==0.5))
    # ggplot(data=comparison.filter(pl.col('horizon')=='2'))
    ggplot(data=comparison)
    + geom_line(aes(x='target_end_date',y='value'),color='blue')
    + geom_line(aes(x='target_end_date', y='value_right'), color='red', linetype='dashed')
    + geom_point(aes(x='target_end_date', y='diff'), color='green')
    + facet_grid(x='horizon',y='quantile',scales='free_y')
)

## Model Ensemble - Trained

*Non-Robust trained*: Use past performance of component forecast to either include it or not in the ensemble, and/or assign weights, doing a **weighted mean**.

*Robust trained*: Use past performance of component forecast to either include it or not in the ensemble, and/or assign weights, doing a **weighted median**.


**Em andamento**